In [14]:
import pandas as pd

In [15]:
productoras_df = pd.read_csv('./csvs/Productora.csv')
sagas_df = pd.read_csv('./csvs/Saga.csv')
juegos_df = pd.read_csv('./csvs/Juego.csv')
usuarios_df = pd.read_csv('./csvs/Usuario.csv')
partidas_df = pd.read_csv('./csvs/Partida.csv')
biblioteca_df = pd.read_csv('./csvs/Biblioteca.csv')

In [16]:
def create_node_queries(df, label, id_column):
    queries = []
    for _, row in df.iterrows():
        properties = ', '.join([f'{col}: {repr(val)}' for col, val in row.items()])
        queries.append(f"CREATE (:{label} {{{properties}}});")
    return queries

# Function to create relationships
def create_relationship_queries(df, start_label, end_label, rel_type, start_key, end_key):
    queries = []
    for _, row in df.iterrows():
        start_id = row[start_key]
        end_id = row[end_key]
        queries.append(f"MATCH (a:{start_label} {{{start_key}: {start_id}}}), (b:{end_label} {{{end_key}: {end_id}}}) CREATE (a)-[:{rel_type}]->(b);")
    return queries


In [17]:
productora_queries = create_node_queries(productoras_df, 'Productora', 'IdProductora')
saga_queries = create_node_queries(sagas_df, 'Saga', 'IdSaga')
juego_queries = create_node_queries(juegos_df, 'Juego', 'IdJuego')
usuario_queries = create_node_queries(usuarios_df, 'Usuario', 'IdUsuario')
partida_queries = create_node_queries(partidas_df, 'Partida', 'IdJuego')  # Assuming there's no IdPartida

# For Ventas, we need to create nodes with multiple properties
venta_queries = []
for _, row in biblioteca_df.iterrows():
    properties = ', '.join([f'{col}: {repr(val)}' for col, val in row.items()])
    venta_queries.append(f"CREATE (:Venta {{{properties}}});")

# Create relationship queries
productora_saga_rel_queries = create_relationship_queries(sagas_df, 'Productora', 'Saga', 'TIENE', 'IdProductora', 'IdSaga')
saga_juego_rel_queries = create_relationship_queries(juegos_df, 'Saga', 'Juego', 'TIENE', 'IdSaga', 'IdJuego')
juego_usuario_partida_rel_queries = create_relationship_queries(partidas_df, 'Juego', 'Partida', 'TIENE', 'IdJuego', 'IdJuego') + \
                                    create_relationship_queries(partidas_df, 'Usuario', 'Partida', 'TIENE', 'IdUsuario', 'IdUsuario')
juego_usuario_venta_rel_queries = create_relationship_queries(biblioteca_df, 'Juego', 'Venta', 'TIENE', 'IdJuego', 'IdJuego') + \
                                  create_relationship_queries(biblioteca_df, 'Usuario', 'Venta', 'TIENE', 'IdUsuario', 'IdUsuario')

# Combine all queries
all_queries = productora_queries + saga_queries + juego_queries + usuario_queries + partida_queries + venta_queries + \
              productora_saga_rel_queries + saga_juego_rel_queries + juego_usuario_partida_rel_queries + juego_usuario_venta_rel_queries


In [18]:
with open('./neo4j/neo4j_queries.cypher', 'w') as file:file.write('\n'.join(all_queries))
